## **Análise de série de vazões (Xingó)**

### 1 - Leitura dos dados e gerando hidrogramas

In [1]:
import pandas as pd
import numpy as np
import plotly as py
import plotly.io as pio
import plotly.graph_objects as go


import sys
sys.path.insert(0, r'C:\Users\arist\OneDrive\Documentos\UFAL\PIBIC 24-25\lib_clebson\HydroComp')

from files.ons import Ons
from files.ana import Ana
from series.flow import Flow
from series.chuva import Chuva
from series.circular import Circular

In [2]:
file = "ONS_daily_flow.csv"
dados = pd.read_csv(file, index_col=0, parse_dates=True)
dados = pd.DataFrame(dados["XINGO (178)"])

#Filtrando período de análise ()
dados = dados[(dados.index >= "1994-09-01") & (dados.index <= "2018-08-31")]

#Convetendo obj pandas para Series Flow
dados = Flow(pd.DataFrame(dados["XINGO (178)"]))

#Gerando hidrograma
fig, data = dados.plot_hydrogram()
#py.offline.plot(fig, filename='gráficos/hidrograma.html')
pio.show(fig)

#Calculando picos anuais
peaks_max = dados.maximum(station="XINGO (178)")

#Gerando hidrograma destanco os valores de pico
fig, data = peaks_max.plot_hydrogram()
#py.offline.plot(fig, filename='gráficos/histo_max.html')
pio.show(fig)


### **2 - Séries de durações parciais**

#### 2.1 - Separando eventos hidrológicos e calculando métricas

In [3]:
station = "XINGO (178)"

#'stationary' or 'events_by_year'
type_threshold = 'stationary' 

#'flood' or 'drought'
type_event = 'flood'

#'media', 'mediana' or 'autocorrelation' 
type_criterion = 'median'

#Peaks or percentil
value_threshold = 0.75

peaks_sdp = dados.parcial(station, type_threshold, type_event, type_criterion, value_threshold)
peaks_sdp.peaks.head()

,Duration,Start,End,peaks
1995-03-02,142,1994-12-05,1995-04-26,5328.0
1996-01-17,132,1995-11-21,1996-04-01,5481.0
1997-01-24,172,1996-11-29,1997-05-20,7502.0
1998-01-03,98,1997-12-13,1998-03-21,5297.0
1999-03-22,147,1998-11-18,1999-04-14,5985.0


In [4]:
#Armazenando dataframe da SDP calculada
df_sdp = peaks_sdp.peaks

In [110]:
fig, data = peaks_sdp.plot_hydrogram(title='SDP-Xingo')
py.offline.plot(fig, filename='gráficos/histo_sdp_by_year.html')

'gráficos/histo_sdp_by_year.html'

##### Estimando parâmetros de distribuição para magnitude e duração

In [4]:
#Máxima verosimilhança
print('------------- MVS ------------- ')
for variable in ['peaks', 'Duration']:
    print('\n', variable)
    for distribuition in ['Gev', 'GP', 'P3']:
        print(f'{distribuition}: {peaks_sdp.mvs(distribuition, variable)}')
print('------------------------------- \n')

#Momentos-L
print('------------- MML ------------- ')
for variable in ['peaks', 'Duration']:
    print('\n', variable)
    for distribuition in ['Gev', 'GP', 'P3']:
        print(f'{distribuition}: {peaks_sdp.mml(distribuition, variable)}')
print('------------------------------- ')




------------- MVS ------------- 

 peaks
Gev: (-6.564323086328912, 3882.462734405476, 3.0375420909799518)
GP: (-0.16327398505263607, 3881.9241208697904, 4122.4799673721955)
P3: (1.4954174010596093, 7456.075517614276, 2747.858176919848)

 Duration
Gev: (0.025555258610584084, 85.73978682104861, 51.052350229309226)
GP: (-0.19391625610592814, 4.999999997262654, 124.40411139601082)
P3: (0.6879459377211087, 112.25001864532328, 59.37297751010044)
------------------------------- 

------------- MML ------------- 

 peaks
Gev: [-0.2352462611296721, 6034.870155213688, 1620.83987875888]
GP: [-0.006753770914668085, 4502.188466580488, 2973.836978393338]
P3: [1.982012743514436, 7456.075568181818, 2937.9496660470454]

 Duration
Gev: [0.260885080809076, 92.53266136920445, 53.708800488853086]
GP: [-0.9493714577112791, 21.42584722924157, 177.05001108212528]
P3: [0.07870432442987278, 112.25, 54.592236523933465]
------------------------------- 


### **3 - Modelagem das distribuições marginais**

#### **3.1 - Magnitude (Generalizada de Pareto)**

##### Máxima Verossimilhança

In [4]:
#Distribuição cumulativa
title = 'Xingó'
type_function = 'cumulative'
fig_acum_mvs_pareto, data_acum_mvs_pareto = peaks_sdp.plot_distribution(title, type_function, estimador='mvs', distribuition='GP', variable='peaks')
#py.offline.plot(fig, filename='gráficos/distrib_cum_sdp_mvs.html')
#pio.show(fig_acum_mvs_pareto)

#Distribuição densidade-probabilidade
title = 'Xingó'
type_function = 'density'
fig_density_mvs_pareto, data_density_mvs_pareto = peaks_sdp.plot_distribution(title, type_function, estimador='mvs', distribuition='GP', variable='peaks')
#py.offline.plot(fig, filename='gráficos/distrib_density_sdp_mvs.html')
#pio.show(fig_density_mvs_pareto)

##### Momemtos L

In [5]:
#Distribuição cumulativa
title = 'Xingó'
type_function = 'cumulative'
fig_acum_mml_pareto, data_acum_mml_pareto = peaks_sdp.plot_distribution(title, type_function, estimador='mml', distribuition='GP', variable='peaks')
#py.offline.plot(fig, filename='gráficos/distrib_cum_sdp_mvs.html')
pio.show(fig_acum_mml_pareto)

#Distribuição densidade-probabilidade
title = 'Xingó'
type_function = 'density'
fig_density_mml_pareto, data_density_mmml_pareto = peaks_sdp.plot_distribution(title, type_function, estimador='mml', distribuition='GP', variable='peaks')
#py.offline.plot(fig, filename='gráficos/distrib_density_sdp_mvs.html')
pio.show(fig_density_mml_pareto)

##### Verificando parâmetros

In [6]:
#Parâmetros 
print("MML: ", peaks_sdp.mml())

print("MVS", peaks_sdp.mvs())

TypeError: Parcial.mml() missing 2 required positional arguments: 'distribuition' and 'variable'

##### Visualizando distribuições pelos dois estimadores

In [6]:
#Definindo legenda
fig_density_mml_pareto['data'][0].name = "MML"
fig_density_mvs_pareto['data'][0].name = "MVS"

#Ajustando cor dos gráficos
fig_density_mml_pareto['data'][0].line.color = 'rgb(0, 0, 0)'

#Densidade
fig_density = go.Figure(data = fig_density_mvs_pareto['data'] + fig_density_mml_pareto['data'])

fig_density.update_layout(
    title="GP - Densidade Xingó",
    xaxis_title="Vazão (m³/s)",
    showlegend=True,
    font=dict(family='Time New Roman', size=20, color='rgb(0,0,0)')) 

fig_density.show()


#Acumulado

#Definindo legenda
fig_acum_mml_pareto['data'][0].name = "MML"
fig_acum_mvs_pareto['data'][0].name = "MVS"

#Ajustando cor dos gráficos
fig_acum_mml_pareto['data'][0].line.color = 'rgb(0, 0, 0)'

fig_acum = go.Figure(data = fig_acum_mvs_pareto['data'] + fig_acum_mml_pareto['data'])

fig_acum.update_layout(
    title="GP - Acumulado Xingó",
    xaxis_title="Vazão (m³/s)",
    showlegend=True,
    font=dict(family='Time New Roman', size=20, color='rgb(0,0,0)')) 

fig_acum.show()

#### 3.2 - Duração ( Pearson III) 

In [5]:
#Distribuição densidade-probabilidade
title = 'Xingó'
type_function = 'density'
fig_density_mml_pearson, data_density_mmml_pareto = peaks_sdp.plot_distribution(title, type_function, estimador='mml', distribuition='P3', variable='Duration')
#py.offline.plot(fig, filename='gráficos/distrib_density_sdp_mvs.html')
pio.show(fig_density_mml_pearson)

#Distribuição densidade-probabilidade
title = 'Xingó'
type_function = 'cumulative'
fig_density_mml_pearson, data_density_mmml_pareto = peaks_sdp.plot_distribution(title, type_function, estimador='mml', distribuition='P3', variable='Duration')
#py.offline.plot(fig, filename='gráficos/distrib_density_sdp_mvs.html')
pio.show(fig_density_mml_pearson)


In [7]:
#Parâmetros 
print("MML: ", peaks_sdp.mml('P3', 'Duration'))

print("MVS", peaks_sdp.mvs('P3', 'Duration'))

MML:  [0.07870432442987278, 112.25, 54.592236523933465]
MVS (0.6879459377211087, 112.25001864532328, 59.37297751010044)


#### 3.3 - Período de ocorrência (Von-Mises)

In [5]:
#Criando objetivo para manipular os dados de período de ocorrência de cheias 
period = Circular(df_sdp)

#Transformando datas de pico em radianos para a escala mensal
period.circular_date(month_num_start_year_hydrologic=9, unit='degrees')

1995-03-02    179.506849
1996-01-17    135.737705
1997-01-24    143.013699
1998-01-03    122.301370
1999-03-22    199.232877
2000-02-26    175.081967
2001-01-03    122.301370
2002-01-18    137.095890
2003-01-28    146.958904
2004-01-31    149.508197
2005-03-31    208.109589
2006-04-01    209.095890
2007-03-21    198.246575
2008-04-03    211.475410
2009-01-24    143.013699
2010-01-16    135.123288
2011-04-02    210.082192
2012-01-20    138.688525
2012-11-28     86.794521
2013-02-17    166.684932
2014-01-13    132.164384
2014-04-12    219.945205
2014-12-20    108.493151
2015-04-06    214.027397
2016-02-13    162.295082
2017-01-02    121.315068
2017-12-30    118.356164
2018-02-24    173.589041
Name: date_peaks, dtype: float64

##### Módulo Circular

In [5]:
#Criando objetivo para manipular os dados de período de ocorrência de cheias 
period = Circular(df_sdp)

#Transformando datas de pico em radianos para a escala mensal
data_radians_month = period.radians_date(type_date='peak', scale='month')

'''#Salvando o nome dos meses de cada evento
data_radians_month['month'] = data_radians_month['date_peak'].dt.month_name()

#Criando daraframe para armazenar dados para gráfico
df_month = pd.DataFrame()

#Calculando vazão média por mês
df_month['mean_peaks'] = data_radians_month.groupby('month')['peaks'].mean() 

#Calculando quantidade de eventos de cheia em cada mês
df_month['qtd_peaks'] = data_radians_month.groupby('month')['peaks'].count() '''

"#Salvando o nome dos meses de cada evento\ndata_radians_month['month'] = data_radians_month['date_peak'].dt.month_name()\n\n#Criando daraframe para armazenar dados para gráfico\ndf_month = pd.DataFrame()\n\n#Calculando vazão média por mês\ndf_month['mean_peaks'] = data_radians_month.groupby('month')['peaks'].mean() \n\n#Calculando quantidade de eventos de cheia em cada mês\ndf_month['qtd_peaks'] = data_radians_month.groupby('month')['peaks'].count() "

In [ ]:
#Gráfico de 
period.plot_radians_date(type_date='peak', scale='month')

In [32]:
import plotly.express


t = data_radians_month

t['month'] = t['date_peak'].dt.month_name()

t['month_num'] = t['date_peak'].dt.month


#Armazenando frequência de ocorrência  de cada dia dentro do mês
list_freq = []
for radians in t['radians_date']:
    freq = 1
    for radians_compare in t['radians_date']:
        if radians == radians_compare:
            freq += 1
    list_freq.append(freq)

t['freq'] = list_freq


#t = t.sort_values(['month_num', 'freq'])


add = {'date_peak':['01-'+str(mes)+'-2000' for mes in range(1, 13)],
       'radians_date':12*[0],
       'Duration':12*[0],
       'peaks':12*[0],
       'month':['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October','November', 'December'],
       'freq':12*[0],
       'month_num':[month for month in range(1, 13)]}

teste_df = pd.concat([pd.DataFrame(add), t]).sort_values(by=['month_num', 'freq'])

fig = px.bar_polar(teste_df, r="freq", theta="month", 
                   color='peaks',
                   color_continuous_scale= px.colors.sequential.Blues)


fig.update_layout(
    title=dict(text='Distribuição eventos de cheias'),
    font=dict(family='Time New Roman', size=18, color='black'),
    polar_radialaxis_ticksuffix='',
    polar_angularaxis_rotation=90,
    polar_angularaxis_tick0=5,
)


fig.show()


#### FIM TESTE